In [1]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
# Load all file ends with .md
loader = DirectoryLoader('data/', glob="*.md", show_progress=True, loader_cls=UnstructuredMarkdownLoader)
# Break documents into smaller chunks, RecursiveCharacterTextSplitter with chunk_size = 2000 is good
docs = loader.load_and_split(RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 100))
# Embedded to local data vector database
db  = Chroma.from_documents(docs, OllamaEmbeddings(model="mxbai-embed-large",show_progress=True))

OllamaEmbeddings: 100%|██████████| 10/10 [00:15<00:00,  1.54s/it]


In [3]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4}) # Turn VectorStore into a Retriever
local_model = "llama3" # [Phi 3, Mistral, Gamma]
llm = ChatOllama(model=local_model, temperature = 0) # temperature > 1 more creative, random, temperature < 1 deterministic, repetitive

# System prompt
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. 
    Keep the answer concise <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question} 
    Context: {context} 
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "context"],
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [4]:
question = "How can I run a job on Polaris? Write an example job submission script?"
print(question)
print(rag_chain.invoke(question))

How can I run a job on Polaris? Write an example job submission script?


OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  8.96it/s]


To run a job on Polaris, you can use the `qsub` command to submit an interactive or batch job. Here is an example of how to submit an interactive job:

```
qsub -I -l select=1 -l filesystems=home:eagle -l walltime=1:00:00 -q debug -A <project_name>
```

This command requests 1 node for a period of 1 hour in the debug queue, requiring access to the `/home` and `eagle` filesystems. After waiting in the queue for a node to become available, a shell prompt on a compute node will appear. You may then start building applications and testing GPU affinity scripts on the compute node.

To run a batch job, you can use the following command:

```
qsub -l select=1 -l filesystems=home:eagle -l walltime=1:00:00 -q <queue_name> -A <project_name>
```

Replace `<queue_name>` with one of the production queues (e.g. `prod`, `debug`, etc.) and `<project_name>` with your project name.

For GPU-based jobs, you can use the following script to assign GPUs to MPI ranks:

```bash
#!/bin/bash

num_gpus=4

# Need

In [5]:
question = "How to connect to polaris"
print(question)
print(rag_chain.invoke(question))

How to connect to polaris


OllamaEmbeddings: 100%|██████████| 1/1 [00:01<00:00,  1.53s/it]


To connect to Polaris, you can log in using SSH:

ssh <username>@polaris.alcf.anl.gov

Then, type in your password from your CRYPTOCard/MobilePASS+ token.


In [6]:
question = "How many queues are there?"
print(question)
print(rag_chain.invoke(question))

How many queues are there?


OllamaEmbeddings: 100%|██████████| 1/1 [00:01<00:00,  1.79s/it]


There are five production queues: debug, debug-scaling, prod, preemptable, and demand.


In [7]:
question = "Give me more details on these five production queues"
print(question)
print(rag_chain.invoke(question))

Give me more details on these five production queues


OllamaEmbeddings: 100%|██████████| 1/1 [00:01<00:00,  1.65s/it]


According to the provided context, there are five production queues on Polaris:

1. debug
2. prod
3. preemptable
4. demand
5. routing queue (which routes jobs to one of six execution queues: small, medium, large, backfill-small, backfill-medium, and backfill-large)


In [8]:
question = "What are the time max and node max of these five production queues"
print(question)
print(rag_chain.invoke(question))

What are the time max and node max of these five production queues


OllamaEmbeddings: 100%|██████████| 1/1 [00:01<00:00,  1.64s/it]


Based on the provided context, I don't know the time max and node max of these five production queues. The context only provides information about controlling where your job runs, hardware overview, compiling applications, accessing additional software, submitting and running jobs, Lustre file striping, proxy settings, getting assistance, PBS commands, and interactive jobs on compute nodes. It does not mention the time max and node max of specific production queues.


In [9]:
question = "What is Gromacs"
print(question)
print(rag_chain.invoke(question))

What is Gromacs


OllamaEmbeddings: 100%|██████████| 1/1 [00:01<00:00,  1.25s/it]


Gromacs is a versatile package to perform molecular dynamics, i.e. simulate the Newtonian equations of motion for systems with hundreds to millions of particles. It is primarily designed for biochemical molecules like proteins, lipids, and nucleic acids that have a lot of complicated bonded interactions, but since GROMACS is extremely fast at calculating the nonbonded interactions (that usually dominate simulations) many groups are also using it for research on non-biological systems, e.g. polymers.


In [11]:
question = "How to use GROMACS on Polaris"
print(question)
print(rag_chain.invoke(question))

How to use GROMACS on Polaris


OllamaEmbeddings: 100%|██████████| 1/1 [00:01<00:00,  1.24s/it]


To use GROMACS on Polaris, you can follow the instructions provided:

1. Download the latest source code from http://manual.gromacs.org/documentation/2022.1/download.html
2. Unzip the downloaded file using `tar -xzf gromacs-2022.1.tar.gz`
3. Load the necessary modules:
	* `module swap PrgEnv-nvhpc PrgEnv-gnu`
	* `module load cudatoolkit-standalone/11.2.2`
	* `module load gcc/10.3.0`
	* `module load cmake`
4. Create a build directory and configure GROMACS using:
	* `cmake -DCMAKE_C_COMPILER=cc -DCMAKE_CXX_COMPILER=CC ...`
5. Build GROMACS using `make –j 8` and install it using `make install`
6. The installed binary is located at `build/bin/gmx_mpi`

Alternatively, you can use the prebuilt GROMACS binaries available in `/soft/applications/Gromacs/gromacs-2022.1`. A sample PBS script to run GROMACS on Polaris is also provided:

```
#!/bin/sh

PBS -l select=2:system=polaris

PBS -l place=scatter

PBS -l walltime=0:30:00

PBS -q debug

PBS -A PROJECT

PBS -l filesystems=home:grand:eagle

cd